<a href="https://colab.research.google.com/github/SumeyyeSuslu/regression/blob/main/sparkdummyGBR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bu ornek spark uzerinde SparkML tarafından dogrudan desteklenen GBT regression algoritmasını kullanmaktadır.

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [2,924 kB]
Hit:14 http://ppa.launchpad.net/graphics-dr

In [ ]:
      #ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZzZ1`1

In [ ]:
#!tar xvf dummygeometryJSON.tar.gz 
from google.colab import drive
drive.mount('/content/drive')
!tar xvf /content/drive/MyDrive/dummygeometryJSON.tar.gz

dummygeometry.json


In [ ]:
# Spark Session, Pipeline, Functions, and Metrics
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
 
from pyspark.sql.functions import rand
from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import Vector
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
import pyspark.sql.functions as F

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [ ]:
df = spark.read.json('dummygeometry.json')


In [ ]:
df.summary().show()

+-------+-----------------+--------------------+-------------------+------------------+--------------------+----------------------+----------------------+----------------------+------------------+------+-------------------+------------------+
|summary|      CoordinateX|         CoordinateY|        CoordinateZ|          Pressure|Pressure Coefficient|X Component Wall Shear|Y Component Wall Shear|Z Component Wall Shear|               aoa|  beta|               mach|              zone|
+-------+-----------------+--------------------+-------------------+------------------+--------------------+----------------------+----------------------+----------------------+------------------+------+-------------------+------------------+
|  count|           378068|              378068|             378068|            378068|              378068|                378068|                378068|                378068|            378068|378068|             378068|            378068|
|   mean|7.276900698086052|-

In [ ]:
from pyspark.sql.functions import *

# filter rows where zone is 3 or 0 and mach is less than 0.8 and CoordinateZ is less than or equal to 0.26
filtered_df = df.filter((col("zone") == 3) | (col("zone") == 0) & (col("mach") < 0.8) & (col("CoordinateZ") <= 0.26))

In [ ]:
filtered_df.summary().show()

+-------+-------------------+------------------+-------------------+------------------+--------------------+----------------------+----------------------+----------------------+------------------+-----+-------------------+------------------+
|summary|        CoordinateX|       CoordinateY|        CoordinateZ|          Pressure|Pressure Coefficient|X Component Wall Shear|Y Component Wall Shear|Z Component Wall Shear|               aoa| beta|               mach|              zone|
+-------+-------------------+------------------+-------------------+------------------+--------------------+----------------------+----------------------+----------------------+------------------+-----+-------------------+------------------+
|  count|              25331|             25331|              25331|             25331|               25331|                 25331|                 25331|                 25331|             25331|25331|              25331|             25331|
|   mean| 11.303435019840103|0.5

In [ ]:
# Split the dataset randomly into 80% for training and 20% for testing. Passing a seed for deterministic behavior
train_data, test_data = filtered_df.randomSplit([0.8, 0.2], seed = 0)

In [ ]:
vectorAssembler= VectorAssembler(inputCols=['mach','aoa','CoordinateX', 'CoordinateY','CoordinateZ' ],outputCol='features')

gbt = GBTRegressor(featuresCol='features',labelCol='Pressure Coefficient')

In [ ]:

# Define a grid of hyperparameters to test:
#  - maxDepth: maximum depth of each decision tree 
#  - maxIter: iterations, or the total number of trees 
paramGrid = ParamGridBuilder()\
  .addGrid(gbt.maxDepth, [2, 5])\
  .addGrid(gbt.maxIter, [10, 100])\
  .build()
 
# Define an evaluation metric.  The CrossValidator compares the true labels with predicted values for each combination of parameters, and calculates this value to determine the best model.
evaluator = RegressionEvaluator(metricName="rmse", labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol())
 
# Declare the CrossValidator, which performs the model tuning.
cv = CrossValidator(estimator=gbt, evaluator=evaluator, estimatorParamMaps=paramGrid)


In [ ]:

pipeline = Pipeline(stages=[vectorAssembler, cv])

# Train model.  This also runs the indexer.
model = pipeline.fit(train_data)

# Make predictions.
predictions = model.transform(test_data)

# Select example rows to display.
predictions.select("prediction", "Pressure Coefficient", "features").show(5)

# Select (prediction, true label) and compute test error
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

rfModel = model.stages[0]
print(rfModel)  # summary only

+--------------------+--------------------+--------------------+
|          prediction|Pressure Coefficient|            features|
+--------------------+--------------------+--------------------+
| 0.14622437718023906|        0.1499677896|[0.200000003,6.0,...|
|0.027196017252345887|        0.0386430323|[0.6000000238,0.0...|
|  0.3105336134038018|        0.3003864884|[1.0,8.0,10.29580...|
| 0.33042154700795867|        0.3072147369|[1.0,10.0,10.2958...|
| 0.04124814848720778|       -0.0145418299|[0.6000000238,0.0...|
+--------------------+--------------------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 0.0374993
VectorAssembler_cb92d48ac9ff
